In [1]:
# Traffic/ Accidents Monitoring
print("Traffic/ Accidents Monitoring")

Traffic/ Accidents Monitoring


In [4]:
### Use with quandl environment

import twint
import nest_asyncio
from datetime import datetime
import pytz
import pandas as pd
from googlemaps import Client as GoogleMaps
import gmaps
import numpy as np
import warnings

from nlp_process import text_nlp
textNlp=text_nlp()

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 0)

nest_asyncio.apply()

myAPI='xxxxxxxxxxxxxxxxxxxxx'
gmap_latlon = GoogleMaps(myAPI)  # my account API, replace with yours
gmaps.configure(api_key=myAPI)

def available_columns():
    return twint.output.panda.Tweets_df.columns

def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

def LatLon_1(dfIn, colName):
    dfIn['lat'] = ""
    dfIn['lon'] = ""

    for x in range(len(dfIn)):
        dummy=dfIn[colName][x]
        #print(dummy)
        geocode_result = gmap_latlon.geocode(dummy)
        #print(geocode_result)
        try:
            dfIn['lat'][x] = geocode_result[0]['geometry']['location']['lat']
            dfIn['lon'][x] = geocode_result[0]['geometry']['location']['lng']
        except:
            dfIn['lat'][x] = ''
            dfIn['lon'][x] = ''
    return dfIn

def ModifiedColumns(dfAcc, nowStr, to_zone):
    
    dateList=[]
    weightL=[]
    scaleL=[]
    colorList=[]
    for n in dfAcc['date']:
        stringDate = datetime.strptime(n, '%Y-%m-%d %H:%M:%S')
        tweetTime=stringDate.astimezone(to_zone)
        timeString=tweetTime.strftime("%Y-%m-%d %H:%M:%S")
        dateList.append(tweetTime)
        weightL.append(int(timeString[10:13]))
        #print(type(n), ' -- ',type(stringDate))

    #maxW=max(weightL)
    maxW=int(nowStr[10:13])
    #print(maxW)
    weightList=[n/maxW for n in weightL]    
    for n in weightList:
        if(n<=1 and n>=0.9):
            colorList.append('red')
            scaleL.append(8)
        elif(n>=0.5 and n<0.9):
            colorList.append('yellow')
            scaleL.append(4)
        else:
            colorList.append('green')
            scaleL.append(4)

    stringDf=pd.DataFrame(dateList, columns=['datetime'])
    weightDf=pd.DataFrame(weightList, columns=['weight'])
    colorDf=pd.DataFrame(colorList, columns=['color'])
    dfAcc=pd.concat([dfAcc,stringDf,weightDf,colorDf],axis=1)

    dfAcc['lat'].replace('', np.nan, inplace=True)
    dfAcc['lon'].replace('', np.nan, inplace=True)
    dfAcc_1=dfAcc.dropna().copy().reset_index()
    return dfAcc_1



def MakeMap(dfAcc_1):

    locations = dfAcc_1[['lat', 'lon']].copy()
    weights = dfAcc_1['weight'].copy()
    colors=dfAcc_1['color'].values.tolist()

    figure_layout = {
        'width': '1200px',
        'height': '800px',
        'border': '1px solid black',
        'padding': '1px'
    }

    bangkok_coordinates = (13.7464, 100.646)
    fig = gmaps.figure(center=bangkok_coordinates,zoom_level=11, layout=figure_layout)
    
    latList=dfAcc_1['lat'].values.tolist()
    lonList=dfAcc_1['lon'].values.tolist()
    llData=list(zip(latList,lonList))
    labelList=[]
    for n in range(0,len(dfAcc_1)):
        labelList.append({"name":dfAcc_1['username'][n], "datetime":dfAcc_1['datetime'][n],
        "location":(dfAcc_1['lat'][n],dfAcc_1['lon'][n]), "tweet":dfAcc_1['tweet'][n]})
        
    info_box_template = """
    <dl>
    <dt><b><p style="color:red;">username<p></b></dt><dd><p style="color:red;">{name}<p></dd>
    <dt><b><p style="color:red;">datetime<p></b></dt><dd><p style="color:red;">{datetime}<p></dd>
    <dt><b><p style="color:red;">tweet<p></b></dt><dd><p style="color:red;">{tweet}<p></dd>
    </dl>
    """

    plant_info = [info_box_template.format(**plant) for plant in labelList]

    info_box_template_2 = """
    username : {name}
    datetime : {datetime}
    tweet    :{tweet}
    """
    plant_info_2 = [info_box_template_2.format(**plant) for plant in labelList]

    #markerLayer=gmaps.marker_layer(llData, info_box_content=plant_info, hover_text=plant_info_2)
    #fig.add_layer(markerLayer)

    incLayer = gmaps.symbol_layer(
        locations, info_box_content=plant_info, hover_text=plant_info_2, fill_color=colors, stroke_color=colors, scale=5
    )
    fig.add_layer(incLayer)
    return fig

def ProcessTweet(inDf):
    modTweet=[]
    for text in inDf['tweet']:
        modTweet.append(textNlp.ProcessText(text))

    modTweetDf=pd.DataFrame(modTweet, columns=['modTweet'])
    inDf=pd.concat([inDf,modTweetDf],axis=1)
    return inDf

def RemoveWordTweet(inDf):
    modTweet=[]
    for text in inDf['tweet']:
        modTweet.append(textNlp.RemoveWord(text))

    modTweetDf=pd.DataFrame(modTweet, columns=['modTweet'])
    inDf=pd.concat([inDf,modTweetDf],axis=1)
    return inDf

todayUTC=datetime.today()

# dd/mm/YY H:M:S
to_zone = pytz.timezone('Asia/Bangkok')
today=todayUTC.astimezone(to_zone)

todayStr=today.strftime("%Y-%m-%d")+' 00:00:00'
nowStr=today.strftime("%Y-%m-%d %H:%M:%S")

c=twint.Config()

c.Hide_output = True
c.Limit=200
c.Since=todayStr
#c.Since='2020-05-09 00:00:00'
c.Pandas=True
#c.Near="ลาดพร้าว"
c.Search=r"#รถติด OR #อุบัติเหตุ"

twint.run.Search(c)
#available_columns()

displayFlag=0
try:
    df_pd = twint_to_pandas(["date","timezone","place","username", "tweet", "hashtags"])
    #modDf=ProcessTweet(df_pd)
    modDf=RemoveWordTweet(df_pd)
    #dfAcc=LatLon_1(df_pd,'tweet')
    dfAcc=LatLon_1(modDf,'modTweet')
    dfAcc1=ModifiedColumns(dfAcc, nowStr, to_zone)
    fig=MakeMap(dfAcc1)
    displayFlag=1
    display(fig)
except:
    print(' No Data ')
    displayFlag=0



Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1200px'))

In [3]:
if(displayFlag==0):
    print(' NO DATA ')    
else:
    display(dfAcc1[['datetime','tweet']])


,datetime,tweet
0,2020-05-12 15:26:18+07:00,@js100radio รถบรรทุกขนมะม่วง 🥭 อยู่เลนขวา #รถติด #JS100 pic.twitter.com/NtqGNF5cYA – at เขื่อนลำตะคอง (Lam Takong Dam)
1,2020-05-12 13:48:59+07:00,13:40น. #ถนนลาดพร้าว ขาเข้า ที่บริเวณซ.ลาดพร้าว84 รถยนต์ เสียหลักชนต้นไม้ข้างทาง มีผู้บาดเจ็บ การจราจรเคลื่อนตัวช้า #อุบัติเหตุ\n.\nCr.แจ้งเหตุRescueว.4 pic.twitter.com/UPRKyYkANh
2,2020-05-12 13:37:40+07:00,13:37 #ถนนลาดพร้าว ช่วง ซ.ลาดพร้าว71 >แยกโชคชัยสี่ ที่บริเวณซ.ลาดพร้าว84 รถยนต์ เสียหลักชนต้นไม้ข้างทาง มีผู้บาดเจ็บ การจราจรเคลื่อนตัวช้า #อุบัติเหตุ pic.twitter.com/jTjoa5sGNB
3,2020-05-12 12:53:15+07:00,12:46 #ถนนพัฒนาการ ช่วง ซ.พัฒนาการ30 >แยกพัฒนาการ ที่บริเวณจุดกลับรถ ซ.พัฒนาการ 41 รถเก๋ง ชนกับ รถโม่ปูน ขวางช่องทางซ้ายและกลาง การจราจรติดขัด #อุบัติเหตุ #รถติด
4,2020-05-12 11:55:13+07:00,11:54 #ถนนพระราม2 ช่วง บุญถาวร >สน.แสมดำ ที่บริเวณกม.14+800 รถชนกัน 3คัน ขวางช่องทางซ้าย ของทางด่วน การจราจรติดขัด #อุบัติเหตุ #รถติด
5,2020-05-12 11:39:49+07:00,11:39 #ถนนพหลโยธิน จาก แยกลำลูกกา >ที่บริเวณบนสะพานข้ามแยกอนุสรณ์สถาน รถตู้ ชนกับ รถปิคอัพ ขวางช่องทางขวา การจราจรเคลื่อนตัวช้า #อุบัติเหตุ #รถติด
6,2020-05-12 11:35:56+07:00,11:35 #ถนนพหลโยธิน ช่วง อนุสรณ์สถาน >สามแยกลำลูกกา ที่บริเวณซ.พหลโยธิน 64 รถเก๋ง ชนกับ รถปิคอัพ ขวางช่องทางซ้าย การจราจรเคลื่อนตัวช้า #อุบัติเหตุ #รถติด
7,2020-05-12 11:21:24+07:00,RT@Khunpol5 : #ถนนลาดพร้าว ช่วงลาดพร้าว34 รถเมล์ จอดเสียขวางช่องทางซ้าย การจราจรติดขัด #รถติด pic.twitter.com/yQvFtlj0V1
8,2020-05-12 10:46:47+07:00,10:46 #ถนนกาญจนาภิเษก ช่วง ต่างระดับเอกชัย >ต่างระดับพระราม2 ก่อนถึงสะพานข้ามทางรถไฟ เล็กน้อย รถเก๋ง ชนกับ รถปิคอัพ ขวางช่องทางซ้าย ของทางขนาน การจราจรเคลื่อนตัวช้า #อุบัติเหตุ #รถติด
9,2020-05-12 10:05:39+07:00,10.00น. #ถนนรัชดาภิเษก จาก แยกรัชดาพระราม4 >ที่บริเวณหน้าอาคารไทยเบฟฯ รถจยย. ชนกับ รถCRV ขวางช่องทางซ้ายและกลาง บาดเจ็บ 1คน การจราจรเคลื่อนตัวช้า #อุบัติเหตุ\n.\nCr.จร076เก้า1205ฉก.พพ14 pic.twitter.com/2hRQflergo


In [59]:
print("Rain and flood Monitoring")

Rain and flood Monitoring


In [40]:
from datetime import timedelta

def ModifiedColumns_Rain(dfAcc, nowStr, to_zone):
        
    dateList=[]
    weightL=[]
    colorList=[]
    for n in dfAcc['date']:
        stringDate = datetime.strptime(n, '%Y-%m-%d %H:%M:%S')
        tweetTime=stringDate.astimezone(to_zone)
        dateList.append(tweetTime)
        weightL.append(int(n[10:13]))
        #print(type(n), ' -- ',type(stringDate))

    #maxW=max(weightL)
    #maxW=int(todayStr[10:13])
    maxW=int(nowStr[10:13])
    weightList=[n/maxW for n in weightL]    
    for n in weightList:
        if(n<=1 and n>=0.7):
            colorList.append('red')
        elif(n>=0.4 and n<0.7):
            colorList.append('yellow')
        else:
            colorList.append('white')

    stringDf=pd.DataFrame(dateList, columns=['datetime'])
    weightDf=pd.DataFrame(weightList, columns=['weight'])
    colorDf=pd.DataFrame(colorList, columns=['color'])
    dfAcc=pd.concat([dfAcc,stringDf,weightDf,colorDf],axis=1)

    dfAcc['lat'].replace('', np.nan, inplace=True)
    dfAcc['lon'].replace('', np.nan, inplace=True)
    dfAcc_1=dfAcc.dropna().copy().reset_index()

    return dfAcc_1


# dd/mm/YY H:M:S
today=todayUTC.astimezone(to_zone)
diff=today-timedelta(hours=1, minutes=30)

todayStr=today.strftime("%Y-%m-%d")+' 00:00:00'
diffStr=diff.strftime("%Y-%m-%d %H:%M:%S")

c.Hide_output = True
c.Username="@js100radio OR from:@fm91trafficpro OR from:@thaich8news OR from:@Ruamduay OR from:@RDNewsCH3"
#c.Limit=200
c.Since=todayStr
#c.Since='2020-05-09 00:00:00'
#c.Since=diffStr
c.Pandas=True
#c.Near="ลาดพร้าว"
c.Search=r"#ฝนตก OR #พายุฝน OR พายุฝน OR ฝนตกหนัก OR ตกหนัก"

twint.run.Search(c)

displayFlag=0
try:
    df_pd1 = twint_to_pandas(["date","timezone","place","username", "tweet", "hashtags"])
    dfAcc1=LatLon_1(df_pd1,'tweet')
    dfAcc2=ModifiedColumns(dfAcc1, nowStr, to_zone)
    fig1=MakeMap(dfAcc2)
    displayFlag=1
    display(fig1)
except:
    print(' No Data ')
    displayFlag=0

CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1200px'))

In [39]:
if(displayFlag==0):
    print(' NO DATA ')    
else:
    display(dfAcc2[['datetime','tweet']])


,datetime,tweet
0,2020-05-12 10:06:45+07:00,คลิปวินาที #ลมหมุน! พัดเต็นท์ด่านตรวจจุดคัดกรองโควิด – 19 บริเวณด่านตรวจปางมะโอ ถนนสายลำปาง – เด่นชัย แพร่ เขต ต.วังเงิน อ.แม่ทะ จ.ลำปาง ปลิวในพริบตา จนท.วิ่งหนีกระเจิง #ข่าวช่อง8 #ใครๆก็ดูช่อง8กดเลข27 #พายุฝน pic.twitter.com/mYPMmk8oGh
